# Transformer

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print('device:', device)

device: cuda


### Positional Encoding

Encoder

In [49]:
# Relative position encoding

class EncoderPositionalEncoding(nn.Module):
    def __init__(self, d_model):
        super(EncoderPositionalEncoding, self).__init__()
        
        self.fc1 = nn.Linear(4, d_model)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(d_model, d_model)
        
    def forward(self, x, pos):
        pos = self.fc1(pos)
        pos = self.relu(pos)
        pos = self.fc2(pos)
        return x + pos

Decoder

In [50]:
class DecoderPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(DecoderPositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

### Multihead Attention

Encoder

In [51]:
class EncoderMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, max_seq_length):
        super(EncoderMultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.max_seq_length = max_seq_length # Maximum sequence length for positional encoding
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
        self.pe = nn.Parameter(torch.randn(self.d_k, max_seq_length)) # Relative position encoding
        
    def scaled_dot_product_attention(self, Q, K, V, pe, mask=None):
        # Calculate attention scores
        attn_scores = (torch.matmul(Q, K.transpose(-2, -1)) + pe) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        pe = torch.matmul(Q, self.pe[:, self.max_seq_length - K.shape[2]:])
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, pe, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

Decoder

In [52]:
class DecoderMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, max_seq_length):
        super(DecoderMultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.max_seq_length = max_seq_length # Maximum sequence length for positional encoding
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
        self.pe = nn.Parameter(torch.randn(self.d_k, max_seq_length)) # Relative position encoding
        
    def scaled_dot_product_attention(self, Q, K, V, pe, mask=None):
        # Calculate attention scores
        attn_scores = (torch.matmul(Q, K.transpose(-2, -1)) + pe) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        pe = torch.matmul(Q, self.pe[:, self.max_seq_length - K.shape[2]:])
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, pe, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

### Position-wise FFN

In [53]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

### Encoder

In [54]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, max_seq_length):
        super(EncoderLayer, self).__init__()
        self.self_attn = EncoderMultiHeadAttention(d_model, num_heads, max_seq_length)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

### Decoder

In [55]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, max_seq_length):
        super(DecoderLayer, self).__init__()
        self.self_attn = DecoderMultiHeadAttention(d_model, num_heads, max_seq_length)
        self.cross_attn = DecoderMultiHeadAttention(d_model, num_heads, max_seq_length)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

### Transformer

In [56]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_encoder_layers, num_decoder_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_encoder_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_decoder_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, pos, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.encoder_embedding(src))
        tgt_embedded = self.dropout(self.decoder_embedding(tgt))

        
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
    
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# Training the model

In [57]:
import datasets

embeddings = datasets.load_from_disk("embeddings")['embeddings']
labels = datasets.load_from_disk("labels")['labels']

train = datasets.load_from_disk("train_dataset")
test = datasets.load_from_disk("test_dataset")
validate = datasets.load_from_disk("validate_dataset")

In [58]:
d_model = 64
num_heads = 4
num_encoder_layers = 4
num_decoder_layers = 4
d_ff = 256
dropout = 0.1
src_vocab_size = len(embeddings)
tgt_vocab_size = len(labels) + 2
max_seq_length = 200
batch_size = 32

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, 
                          num_heads, num_encoder_layers, num_decoder_layers, 
                          d_ff, max_seq_length, dropout).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
optimizer = optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def create_dataset(data, maxsize = -1):
    emb = data['embeddings']
    pos = data['pos']
    formula = data['formula']
    if maxsize == -1:
        maxsize = len(emb)
    emb = torch.tensor([[emb[e][i][0] if i < len(emb[e]) else 0 for i in range(max_seq_length)] for e in range(maxsize)])
    pos = torch.tensor([[pos[e][i] if i < len(pos[e]) else [0, 0, 0, 0] for i in range(max_seq_length)] for e in range(maxsize)], dtype=torch.float)
    formula = torch.tensor([[len(labels) + 1] + formula[i] + [len(labels) + 1] * (max_seq_length - len(formula[i]) - 1) for i in range(maxsize)])
    return emb, pos, formula

In [ ]:
src_data, pos_data, tgt_data = create_dataset(train)
data_loader = data.DataLoader(data.TensorDataset(src_data, pos_data, tgt_data), batch_size=batch_size, shuffle=True)

In [ ]:
print('Start training...')

transformer.train()

for epoch in range(1):
    for src_data, pos_data, tgt_data in data_loader:
        optimizer.zero_grad()
        src_data = src_data.to(device)
        pos_data = pos_data.to(device)
        tgt_data = tgt_data.to(device)
        
        output = transformer(src_data, pos_data, tgt_data[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Start training...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
with torch.no_grad():
    print(tgt_data.shape)
    print(tgt_data[0:1,:-1])
    print(tgt_data[0:1][:-1])
    output = transformer(src_data[0:1].to(device), pos_data[0:1].to(device), tgt_data[0:1,:-1].to(device)).to(device)
    print(output.shape)
    print(output.contiguous().view(-1, tgt_vocab_size).shape)
    print(tgt_data[0:2, 1:].contiguous().view(-1).shape)
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size).to(device), tgt_data[0:1,1:].contiguous().view(-1).to(device))
    output = output[0]
    print(output)
    ans = []
    for i in range(len(output)):
        ans.append(torch.argmax(output[i]))
    ans = torch.tensor(ans)
    print(ans)
    
    for x in ans:
        print(labels[x - 1], end=' ')
    print()
    
    for x in tgt_data[0]:
        if x == len(labels) + 1: continue
        print(labels[x - 1], end=' ')

torch.Size([16, 200])
tensor([[  0,  42, 234, 515, 282, 282, 431, 297, 374, 515, 157, 515, 150, 431,
         431, 515,  95, 164, 431, 468, 343, 515,  61, 374, 515, 220, 431, 515,
         217, 431, 431, 251,  95,  61, 374, 515,  95, 431, 515, 468, 282, 431,
         344, 199,  42, 234, 515, 356, 431, 297, 468, 343, 515,  61, 374, 515,
         220, 431, 515, 217, 431, 431, 157, 515, 150, 431, 251,  95, 208, 374,
         515, 490, 431, 515, 468, 282, 431, 344,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,  